# Overview
This tutorial walks through a script that trains a feedforward orthography-to-phonology model.

## Dependencies
We always start by importing some libraries the script is dependent upon. Here, we need `json`, `random`, and `time`. Additionally, we need our user-defined modules `learner` and `examples`.

In [2]:
import json
import random
import time
# user-defined:
from learner import setupLearner
import examples

## Configuration
Any model you specify requires certain configuration specs. We typically call these "hyperparameters". We can talk more about why we call them that (as opposed to say "parameters") sometime later. For now, just understand that some of the ingredients to running the model are certain specifications (values of one kind or another) to make it run.

In [3]:
# We call the configuration file cfg, by convention
with open('params.json', 'r') as f:
    cfg = json.load(f)

## Training environment
We always need inputs and outputs, and these patterns can be thought of as the _training environment_. Sometimes, we have multiple inputs and multiple outputs, depending on which aspects of perception and cognition we are interested in. For now, we just have orthography and phonology.

In [4]:
orthography = examples.load('../inputs/orth.csv', simplify=True)
phonology = examples.load('../inputs/phon.csv', simplify=True)
words = [x.strip() for x in open('../inputs/words.csv','r').readlines()]

../inputs/orth.csv
../inputs/phon.csv


## Setting up the learner
In order to train the model, you have to initiate the learner. This involves giving the learner object (however you've defined it) the hyperparameters it needs to run. Here those values have been minimized to the hidden layer size, the maximum number of iterations, the input patterns, the output patterns, and those inputs and outputs we'd like to test at the end of training (in this case, all of them; test set = training set). We also save the start time to a variable so that at the end of training we can calculate the total training time.

In [5]:

T = []

start_time = time.time()
training_environment = random.sample(range(len(orthography)), cfg['training_size'])
T.append(training_environment)
learner, model = setupLearner(
    hidden_layer_sizes=cfg['hidden_size'],
    max_iter=cfg['max_iter'],
    input_patterns=orthography,
    target_patterns=phonology,
    test_set = training_environment)


## "Fitting" the model
Now that we've saved the learner (and the untrained model) to variables, we can fit the model to the environment we've specified. There are different ways to configure the learner and model objects, but we've constructed a relatively simple one here. It might be confusing as to why `learner` and `model` exist as different objects, but we'll explain that more later. Just know that when you fit the `learner` to the environment, the state of the model is saved in `model`. Typically the method used to train the learner is called `fit()`. This is a conventional method name, and we use it here as well. We also print the time elapsed at the end of training, just to keep tabs on how long it is taking.

In [6]:
learner.fit(training_environment)
end_time = time.time()
print(round((end_time-start_time)/60, 2), "minutes elapsed since start of learning")

0.14 minutes elapsed since start of learning


/home/mcb/miniconda3/envs/python3_env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Saving the test data
After the model is set, we can test it and save the data. Here we've done the absolute simplest thing: test the learner on all the patterns at the end of training. We then save this to an output file, and label the file with the date and time that the model runs (`dateID`). The code for testing looks complicated, but it really is just splitting up the training items and identifying which are train and which are test items. In our case here the test items are the same as the train items. We'll do fancier things later.

In [7]:

dateID = time.asctime().replace(' ', '_')

with open(str('../outputs/' + dateID + '.csv'), 'w') as f:
    for i,w in enumerate(words):
        f.write("{word:s},".format(word=w))
        isTrainingItem = [i in T[j] for j in range(len(T))]
        f.write("{vec:s}\n".format(vec = ','.join([str(int(tf)) for tf in isTrainingItem])))

## All done
Now you've trained and tested your model. You can move on to do the important stuff: analyze what the model knows and think about why.